# Generative use case: SQL Code gen

In this notebook, we will explore how we can generate SQL from natural language. 

There are several options to acheive this, such as:
1. [Langchain](https://js.langchain.com/docs/modules/chains/other_chains/sql)
1. [Guardrails](https://shreyar.github.io/guardrails/use_cases/text2sql/text2sql/)
1. [Guidance](https://github.com/microsoft/guidance)

> Note - This can be risky, requiring thorough checks for any SQL injection or Data Exfiltration attacks.

#### 1. Import Semantic Kernel dependencies

In [5]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230711.7-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.Orchestration;

Installed Packages Microsoft.SemanticKernel, 0.17.230711.7-preview

#### 2. Initialize kernel

In [7]:
var builder = new KernelBuilder();

var (_, model, azureEndpoint, apiKey, org, bingApiKey) = Settings.LoadFromFile();
builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);

IKernel kernel = builder.Build();

#### 3. Create a Semantic Function (Prompt Template) with a one-shot example

In [3]:
string skPrompt = @"
{{$input}}

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: How much did a user spend at restaurants last month
SQLQuery: SELECT 
                u.user_id,
                u.first_name,
                u.last_name,
                SUM(t.transaction_amount) AS total_spent
            FROM 
                users u
            JOIN 
                transactions t ON u.user_id = t.user_id
            WHERE 
                t.transaction_date >= DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month')
                AND t.transaction_date < DATE_TRUNC('month', CURRENT_DATE)
                AND t.transaction_type = 'Restaurant'
            GROUP BY 
                u.user_id, u.first_name, u.last_name; 
Only use the following tables:

Table info:
{{$tableInfo}}

Question: {{$input}}.
";

// create semantic function
var textToSQL = kernel.CreateSemanticFunction(skPrompt, maxTokens: 200, temperature: 0, topP: 0.5);
Console.WriteLine(textToSQL.ToString());

{"Name":"func4e7e959b9d6f4c1fbedbc17472c3798f","SkillName":"_GLOBAL_FUNCTIONS_","Description":"Generic function, unknown purpose","IsSemantic":true,"RequestSettings":{"Temperature":0,"TopP":0.5,"PresencePenalty":0,"FrequencyPenalty":0,"MaxTokens":200,"StopSequences":[],"ResultsPerPrompt":1,"ChatSystemPrompt":"Assistant is a large language model.","TokenSelectionBiases":{}},"Parameters":[{"Name":"input","Description":"","DefaultValue":""},{"Name":"tableInfo","Description":"","DefaultValue":""}]}


#### 3. Create Context variables

In [4]:
var tableInfo = @"
    Users table has columns: user_id, first_name, last_name, and email.
    Transactions table has columns: transaction_id, user_id, transaction_amount, transaction_date, and transaction_type.
";

// context variables
var context = kernel.CreateNewContext();
context["dialect"] = "PostgreSQL";
context["tableInfo"] = tableInfo;

#### 4. Generate SQL from natural language

In [5]:
context["input"] = "Most expensive purchase in the last year";
var sql = await textToSQL.InvokeAsync(context);
Console.WriteLine(sql);

SQLQuery: SELECT 
                u.first_name,
                u.last_name,
                t.transaction_amount,
                t.transaction_date
            FROM 
                users u
            JOIN 
                transactions t ON u.user_id = t.user_id
            WHERE 
                t.transaction_date >= DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 year')
                AND t.transaction_date < DATE_TRUNC('year', CURRENT_DATE)
            ORDER BY 
                t.transaction_amount DESC
            LIMIT 1;

This query will return the first name, last name, transaction amount, and transaction date of the user who made the most expensive purchase in the last year. The results will be ordered by transaction amount in descending order, and only the top result will be returned using the LIMIT keyword.
